In [ ]:
!pip3 install openpyxl

In [ ]:
import numpy as np
import pandas as pd
import math
import random
from collections import Counter
import itertools

n = 7
pown = 2 ** n
VTnums = [1,1,2,2,4,6,10,16,30,52,94,172,316,586,1096, 2048, 3856, 7286, 13798, 26216, 49940, 95326, 182362, 349536, 671092, 1290556, 2485534, 4793492, 9256396, 17895736, 34636834, 67108864, 130150588, 252645136, 490853416]


def del4nat(binseq, n):
  SET = set()
  for i in range(0, n):
    up  = binseq // (2 ** (i+1))
    low = binseq % (2 ** i)
    join = up * (2 ** i ) + low
    SET.add(join)
  return SET

def ins4nat(binseq,n):
  SET = set()
  for i in range(0,n):
    up  = binseq // (2 ** (i))
    low = binseq % (2 ** i)
    ins1 = up*2 + 1
    ins0 = up*2
    ins1 = ins1*(2**i) + low
    ins0 = ins0*(2**i) + low
    SET.add(ins1)
    SET.add(ins0)
  return SET

def list2density(liss,n):
  code_list = list(range(2**n))
  count = 0
  for i in liss:
    for d in del4nat(i,n):
      try:
        code_list.remove(d)
      except ValueError:
        count += 1
  return count

from math import ceil, log2
def check_sum(i):
    if i == 0:
        return 0
    bin_len = ceil(log2(i))+1
    sum = 0
    for j in range(bin_len):
        sum += (j+1)*((i//(2**j))%2)
    return sum

bin_string_dict = {}
for m in range(15):
    bin_strings = []
    for x in range(2**m):
        bin_strings.append(bin(x)[2:].zfill(m))
    bin_string_dict[m] = bin_strings

def ham_weight(x):
    count = 0
    for i in range(len(x)):
        if int(x[i]) == 1:
            count += 1
    return count

def wt_int(x):
    count = 0
    for i in range(n):
        count += (x//(2**i))%2
    return count

def runs_int(x):
    count = 1
    for i in range(n-1):
        a = (x//(2**i))%2
        b = (x//(2**(i+1)))%2
        if a != b:
             count += 1
    return count
a = 127
print(wt_int(a))
print(runs_int(a))

def VTclass(v,i):
    c = check_sum(v)%(i+1)
    return c


7
1


In [ ]:
def gen_probs(n):
  cslim = int(n*(n+1)/2)+1
  cs = dict()
  for x in range(cslim):
      cs[x] = []
      for r in list(range(2**n)):
          if check_sum(r) == x:
              cs[x].append(r)
  cscounts = [len(cs[x]) for x in range(cslim)]
  cs_count_combos = []
  for i in range(cslim):
    row = []
    for j in range(cslim):
      col = cscounts[i]*cscounts[j]
      row.append(col)
    cs_count_combos.append(row)
  cs_count_combos = np.array(cs_count_combos)
  #print(cs_count_combos)
  #print(cs[14])
  #print(cs[15])
  nparray = []
  for x in range(cslim):
    row = []
    for z in range(cslim):
      col = 0
      for w in cs[x]:
          dels1 = set(del4nat(w,n))
          for y in cs[z]:
              dels2 = set(del4nat(y,n))
              if dels1 & dels2:
                  #print(y,end=' ')
                  col += 1
      row.append(col)
    #print(row)
    nparray.append(row)
  nparray = np.array(nparray)
  #print(nparray)
  probs = np.divide(nparray,cs_count_combos)
  #probs = np.around(probs,decimals = 2)
  return probs

def count_lemma(n,k):
   if n >= k:
      return math.comb(n,k)
   else:
      return 1

def gen_probs_dependent(n,k1,k2):
  #generates probabilities of deletion-sharing by combos of words, not just single
  cslim = int(n*(n+1)/2)+1
  cs = dict()
  for x in range(cslim):
      cs[x] = []
      for r in list(range(2**n)):
          if check_sum(r) == x:
              cs[x].append(r)
  cscounts = [len(cs[x]) for x in range(cslim)]
  csk1counts = [count_lemma(cscounts[x],k1) for x in range(cslim)]
  csk2counts = [count_lemma(cscounts[x],k2) for x in range(cslim)]
  cs_count_combos = []
  cs_kcount_combos = []
  for i in range(cslim):
    row = []
    rowk = []
    for j in range(cslim):
      col = cscounts[i]*cscounts[j]
      colk = csk1counts[i]*csk2counts[j]
      row.append(col)
      rowk.append(colk)
    cs_count_combos.append(row)
    cs_kcount_combos.append(rowk)
  cs_count_combos = np.array(cs_count_combos)
  cs_kcount_combos = np.array(cs_kcount_combos)
  nparray = []
  for x in range(cslim):
    row = []
    if k1 <= len(cs[x]):
      delsk1 = [list(a) for a in itertools.combinations(cs[x],k1)]
    else:
      delsk1 = [cs[x]]
    for z in range(cslim):
      col = 0
      if k2 <= len(cs[z]):
        delsk2 = [list(a) for a in itertools.combinations(cs[z], k2)]
      else:
        delsk2 = [cs[z]]
      for ycombo in delsk1:
        coladd1 = 0
        for wcombo in delsk2:
          coladd2 = 0
          for y in ycombo:
            dels1 = set(del4nat(y,n))
            for w in wcombo:
              dels2 = set(del4nat(w,n))
              if dels1 & dels2:
                coladd2 = 1
          coladd1 += coladd2
        col += coladd1
      row.append(col)
    nparray.append(row)
  nparray = np.array(nparray)
  cs_kcount_combos[cs_kcount_combos == 0] = 1
  probs = np.divide(nparray,cs_kcount_combos)
  #probs = np.around(probs,decimals = 2)
  return probs


def gen_probs_trios(n):
  #modified to compare 3 at a time
  cslim = int(n*(n+1)/2)+1
  cs = dict()
  for x in range(cslim):
      cs[x] = []
      for r in list(range(2**n)):
          if check_sum(r) == x:
              cs[x].append(r)
  cscounts = [len(cs[x]) for x in range(cslim)]
  cs_count_combos = []
  for i1 in range(cslim):
    ax1 = []
    for i2 in range(cslim):
      ax2 = []
      for i3 in range(cslim):
        ccount = dict(Counter([i1,i2,i3]))
        cval = np.prod([count_lemma(cscounts[x],y) for (x,y) in ccount.items()])
        ax2.append(cval)
      ax1.append(ax2)
    cs_count_combos.append(ax1)
  cs_count_combos = np.array(cs_count_combos)
  #print(f'cs_c_combs {cs_count_combos}')
  #print(cs[14])
  #print(cs[15])
  npbigarray = []
  for x1 in range(cslim):
    ax1 = []
    for x2 in range(cslim):
      ax2 = []
      for x3 in range(cslim):
        ax3 = trio_deletion_count(n,x1,x2,x3)
        ax2.append(ax3)
      ax1.append(ax2)
    npbigarray.append(ax1)
  npbigarray = np.array(npbigarray)
  #print(nparray)
  probs = np.divide(npbigarray,cs_count_combos)
  #probs = np.around(probs,decimals = 2)
  return probs

def cs_fun_trios(n):
  cslim = int(n*(n+1)/2)+1
  cs = dict()
  for x in range(cslim):
    cs[x] = []
    for r in list(range(2**n)):
      if check_sum(r) == x:
        cs[x].append(r)
  return cs


def trio_deletion_count(n,x1,x2,x3):
  cs = cs_fun_trios(n)
  counter = 0
  if x1 == x2 and x3 != x1:
    #print(f'nondistinct x1 {x1} x2 {x2} x3 {x3}')
    ind_len = len(cs[x1])
    for i in range(ind_len):
      w1 = cs[x1][i]
      dels1 = set(del4nat(w1,n))
      for j in range(i,ind_len):
        w2 = cs[x1][j]
        dels2 = set(del4nat(w2,n))
        for w3 in cs[x3]:
          dels3 = set(del4nat(w3,n))
          if ((dels1 & dels2) or (dels1 & dels3) or (dels2 & dels3)) and (w1 != w2) and (w1 != w3) and (w2 != w3):
            #print(f'x{w1}y{w2}z{w3}',end=' ')
            counter += 1
  elif x1 == x3 and x2 != x1:
    ind_len = len(cs[x1])
    for i in range(ind_len):
      w1 = cs[x1][i]
      dels1 = set(del4nat(w1,n))
      for j in range(i,ind_len):
        w3 = cs[x1][j]
        dels3 = set(del4nat(w3,n))
        for w2 in cs[x2]:
          dels2 = set(del4nat(w2,n))
          if ((dels1 & dels2) or (dels1 & dels3) or (dels2 & dels3)) and (w1 != w2) and (w1 != w3) and (w2 != w3):
            #print(f'x{w1}y{w2}z{w3}',end=' ')
            counter += 1
  elif x2 == x3 and x3 != x1:
    ind_len = len(cs[x3])
    for i in range(ind_len):
      w3 = cs[x3][i]
      dels3 = set(del4nat(w3,n))
      for j in range(i,ind_len):
        w2 = cs[x2][j]
        dels2 = set(del4nat(w2,n))
        for w1 in cs[x1]:
          dels1 = set(del4nat(w1,n))
          if ((dels1 & dels2) or (dels1 & dels3) or (dels2 & dels3)) and (w1 != w2) and (w1 != w3) and (w2 != w3):
            #print(f'x{w1}y{w2}z{w3}',end=' ')
            counter += 1
  elif x1 == x2 and x2 == x3:
    ind_len = len(cs[x1])
    for i in range(ind_len):
      w1 = cs[x1][i]
      dels1 = set(del4nat(w1,n))
      for j in range(i,ind_len):
        w2 = cs[x1][j]
        dels2 = set(del4nat(w2,n))
        for k in range(j,ind_len):
          w3 = cs[x1][k]
          dels3 = set(del4nat(w3,n))
          if ((dels1 & dels2) or (dels1 & dels3) or (dels2 & dels3)) and (w1 != w2) and (w1 != w3) and (w2 != w3):
            #print(f'x{w1}y{w2}z{w3}',end=' ')
            counter += 1
  else: #distinct xs
    #print(f'distinct x1 {x1} x2 {x2} x3 {x3}')
    for w1 in cs[x1]:
      dels1 = set(del4nat(w1,n))
      for w2 in cs[x2]:
        dels2 = set(del4nat(w2,n))
        for w3 in cs[x3]:
          dels3 = set(del4nat(w3,n))
          if ((dels1 & dels2) or (dels1 & dels3) or (dels2 & dels3)) and (w1 != w2) and (w1 != w3) and (w2 != w3):
            #print(f'x{w1}y{w2}z{w3}',end=' ')
            counter += 1
  return counter


## convert your array into a dataframe
df1 = pd.DataFrame(gen_probs(3))
df2 = pd.DataFrame(gen_probs_dependent(3,1,2))
## save to xlsx file

filepath1 = 'checksum_empirical_probs3trash.xlsx'
filepath2 = 'checksum_empirical_k12probs3trash.xlsx'

df1.to_excel(filepath1, index=False)
df2.to_excel(filepath2, index=False)

In [ ]:
n=3
cslim = int(n*(n+1)/2)+1
cs = dict()
for x in range(cslim):
    cs[x] = []
    for r in list(range(2**n)):
        if check_sum(r) == x:
            cs[x].append(r)
ab = [check_sum(r) for r in list(range(2**n))]
print(ab)
bc = [list(a) for a in itertools.combinations(ab,3)][10:100]
print(bc)
probs_trios = gen_probs_trios(n)
print([probs_trios[x[0]][x[1]][x[2]] for x in bc])

[0, 1, 2, 3, 3, 4, 5, 6]
[[0, 2, 6], [0, 3, 3], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 4, 5], [0, 4, 6], [0, 5, 6], [1, 2, 3], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 3, 3], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 4, 5], [1, 4, 6], [1, 5, 6], [2, 3, 3], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 4, 5], [2, 4, 6], [2, 5, 6], [3, 3, 4], [3, 3, 5], [3, 3, 6], [3, 4, 5], [3, 4, 6], [3, 5, 6], [3, 4, 5], [3, 4, 6], [3, 5, 6], [4, 5, 6]]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
n=5
ccount = dict(Counter([0,3,3]))
print(ccount.keys())
print(ccount)
cslim = int(n*(n+1)/2)+1
cs = dict()
for x in range(cslim):
    cs[x] = []
    for r in list(range(2**n)):
        if check_sum(r) == x:
            cs[x].append(r)
cscounts = [len(cs[x]) for x in range(cslim)]
print(cscounts)
cs_count_combos = []
for i1 in range(cslim):
    ax1 = []
    for i2 in range(cslim):
        ax2 = []
        for i3 in range(cslim):
            ccount = dict(Counter([i1,i2,i3]))
            cval = np.prod([count_lemma(cscounts[x],y) for (x,y) in ccount.items()])
            ax2.append(cval)
        ax1.append(ax2)
    cs_count_combos.append(ax1)
cs_count_combos = np.array(cs_count_combos)
print(cs_count_combos[0][6][6])

dict_keys([0, 3])
{0: 1, 3: 2}
[1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 1, 1, 1]
3


In [ ]:
def max_cs_group(n):
  return max(list(dict(Counter([check_sum(i) for i in range(2**n)])).values()))

def sim_success_dependent(n,itrs = 1000):
  size = VTnums[n]+1
  problist = []
  probs_dependent = dict()
  for k1 in range(1,max_cs_group(n)+1):
    probs_dependent[k1] = dict()
    for k2 in range(1,max_cs_group(n)+1):
      probs_dependent[k1][k2] = gen_probs_dependent(n,k1,k2)
  for i in range(itrs):
    luckycode = random.sample(range(2**n),size)
    #print(f'luckycode {luckycode}')
    luckysums = [check_sum(c) for c in luckycode]
    luckycounts = dict(Counter(luckysums))
    #print(f'luckysums {luckysums}')
    #print(f'luckycounts {luckycounts}')
    luckyprob = 1
    values = list(luckycounts.values())
    keys = list(luckycounts.keys())
    for xi in range(len(keys)):
      for yi in range(xi+1,len(keys)):
        luckyprob *= (1 - probs_dependent[values[xi]][values[yi]][keys[xi]][keys[yi]])
    problist.append(luckyprob)
  return np.average(problist)


def sim_success_trios(n,itrs = 1000):
  size = VTnums[n]+0 #change to +1 for counterexample
  problist = []
  probs_trios = gen_probs_trios(n)
  for i in range(itrs):
    luckycode = random.sample(range(2**n),size)
    #print(f'luckycode {luckycode}')
    luckysums = [check_sum(c) for c in luckycode]
    luckytrios = [list(a) for a in itertools.combinations(luckysums,3)]
    #print(f'luckysums {luckysums}')
    #print(f'luckycounts {luckycounts}')
    luckyprob = 1
    for t in luckytrios:
      luckyprob *= (1 - probs_trios[t[0]][t[1]][t[2]])
    problist.append(luckyprob)
  return np.average(problist)

In [ ]:
def log_lemma(x):
  if x == 0:
    return 'und.'
  else:
    return np.log10(x)
for i in range(3,8):
  s = sim_success_trios(i)
  t = math.comb(2**i,VTnums[i]+0) #change to +1 for counterexample
  factor = s*t
  #1/e < .37
  print('n =', i,'single countereg prob.',s,'#codes', t, 'total prob',1 - .37**factor)

n = 3 single countereg prob. 1.0 #codes 28 total prob 0.9999999999991879
n = 4 single countereg prob. 0.001140625 #codes 1820 total prob 0.873055535897545
n = 5 single countereg prob. 2.0073028241922898e-11 #codes 906192 total prob 1.808530281843801e-05
n = 6 single countereg prob. 3.194042455734877e-44 #codes 151473214816 total prob 0.0
n = 7 single countereg prob. 1.2543111218893453e-124 #codes 93343021201262177400 total prob 0.0


In [ ]:
s7 = sim_success_trios(7)
print('at n =', 7,'single code bigger than VT(0) prob.', s7, 'countereg prob',1 - .37**factor)

at n = 7 single code bigger than VT(0) prob. 6.94204107962179e-96 countereg prob 0.0


In [ ]:
n = 3
probs_dependent = dict()
for k1 in range(1,max_cs_group(n)+1):
  probs_dependent[k1] = dict()
  for k2 in range(1,max_cs_group(n)+1):
    probs_dependent[k1][k2] = gen_probs_dependent(n,k1,k2)

for k1 in range(1,n):
  for k2 in range(1,n):
    print(k1, k2, probs_dependent[k1][k2])

1 1 [[1.  1.  1.  0.5 0.  0.  0. ]
 [1.  1.  1.  1.  1.  0.  0. ]
 [1.  1.  1.  1.  1.  1.  0. ]
 [0.5 1.  1.  0.5 1.  1.  0.5]
 [0.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.5 1.  1.  1. ]]
1 2 [[1.  1.  1.  1.  0.  0.  0. ]
 [1.  1.  1.  1.  1.  0.  0. ]
 [1.  1.  1.  1.  1.  1.  0. ]
 [0.5 1.  1.  1.  1.  1.  0.5]
 [0.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  1.  1.  1.  1. ]]
2 1 [[1.  1.  1.  0.5 0.  0.  0. ]
 [1.  1.  1.  1.  1.  0.  0. ]
 [1.  1.  1.  1.  1.  1.  0. ]
 [1.  1.  1.  1.  1.  1.  1. ]
 [0.  1.  1.  1.  1.  1.  1. ]
 [0.  0.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.5 1.  1.  1. ]]
2 2 [[1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1.]]
